In [1]:
import numpy as np
import warnings
from my_modules.helpers import *
import my_modules.variables as vb
import my_modules.direct_srv as ds
import my_modules.analyticalsrv_func as asrv
np.set_printoptions(suppress=True)

In [3]:
ps = np.array([0.2791386,  0.28731193, 0.43354946])
qs = np.array([0.39314196, 0.3746973,  0.23216074])
#### derived variables #####
srv_states = 3
p_states = ps.shape[0]
q_states = qs.shape[0]

In [4]:
#### analytically calculated ####
master_mat = np.concatenate((np.kron(np.eye(p_states), ps), np.kron(qs, np.eye(q_states))), axis = 0)
pinv = np.linalg.pinv(master_mat)
null_mat = np.eye(9) - np.dot(pinv, master_mat)
pinv_nm = null_mat

In [14]:
#### loop variables ####
prev_srvs = []
prev_sum = 0
srvs = []
# r_srvs, r_srv= vb.give_rick_srvs(ps,qs)
# bs, target_marginal = vb.give_rick_bs(r_srv,ps,qs)
# ws = vb.give_rick_ws(r_srvs, bs, pinv, pinv_nm)
# print_info(r_srv, qs,ps)

target_marginal = ps[::-1]
bs = np.array([target*np.ones(p_states+q_states) for target in target_marginal])
ws = [np.zeros(p_states*q_states)]*3
ideals = [np.array([1,0,0,0,1,0,0,0,1]),
          np.array([0,1,0,0,0,1,1,0,0]),
          np.array([0,0,1,1,0,0,0,1,0])]
for i in range(3):
    ws[i] = np.dot(pinv_nm, (ideals[i] - target_marginal[i]))
i_srv = asrv.generate_simple_srv(ps,qs,bs,ws,pinv,null_mat,master_mat)
# print_info(i_srv, qs, ps)



In [15]:
for i in range(0, srv_states-1):
    #we find the solution assuming the solution['free_params'] = 0 and then solution['put_one_in'] = 1:
    b,w = bs[i], ws[i]
    solution = np.dot(pinv, b) + np.dot(null_mat, w)
    solution[abs(solution)<1e-8] = 0
    solution[abs(1-solution)<1e-8] = 1
    

    #The above step only works when the target_marginal is chosen wisely:
    assert np.all(abs(np.dot(master_mat, solution)-bs[i])<1e-7)
    

    prev_sum += solution
    prev_srvs.append(solution)
    srvs.append(solution.reshape(q_states,p_states))
prev_srvs.append(1-prev_sum)



In [16]:
#calculate the last element of the srvs##
A, B, C = impurify(srvs[0], srvs[1], (1-prev_sum).reshape(3,3), do_it = 0)
srvs.append(C)
## ----------------------------------##

#### finallyyyy ####
srv = np.concatenate([matrix[..., np.newaxis] for matrix in srvs], axis=2)
srv[abs(srv)< 1e-13] = 0
##----------------#
marginals = calculate_marginals(srv, ps, qs).reshape(6,3)
print("\nFinal marginals:", marginals,'\n')
any_mismatch = np.any(abs(marginals - marginals[0])>1e-2, axis=1)
generate_uhoh(not np.any(any_mismatch))

mutual_info, entropy = calculate_mutual_info(srv, ps,qs, srv_states)
print("\nMutual info and entropy:", mutual_info, entropy, "\n")

print("\nSrv:\n",srv, "\n ps, and qs, end_marginal:", repr(ps), ", ", repr(qs), ", ", repr(marginals[0]))
print_info(srv, ps, qs)


Final marginals: [[0.43354946 0.28731193 0.2791386 ]
 [0.43354946 0.28731193 0.27913861]
 [0.43354946 0.28731193 0.2791386 ]
 [0.43354946 0.28731193 0.27913861]
 [0.43354946 0.28731193 0.2791386 ]
 [0.43354946 0.28731193 0.27913861]] 


Mutual info and entropy: nan 1.5535867470686835 


Srv:
 [[[ 1.07590836  0.11716246 -0.19307082]
  [ 0.09597224  0.93628875 -0.03226099]
  [ 0.24368138 -0.0332132   0.78953181]]

 [[ 0.06755768 -0.00817012  0.94061244]
  [ 1.08713559 -0.18367212  0.09653653]
  [ 0.23606182  0.78967626 -0.02573808]]

 [[-0.06352901  1.05233889  0.01119012]
  [-0.04965536 -0.0515204   1.10117576]
  [ 1.07380928  0.01929642 -0.0931057 ]]] 
 ps, and qs, end_marginal: array([0.2791386 , 0.28731193, 0.43354946]) ,  array([0.39314196, 0.3746973 , 0.23216074]) ,  array([0.43354946, 0.28731193, 0.2791386 ])


AssertionError: average conditional entropy should be non-negative